<a href="https://colab.research.google.com/github/Hellblazer99/AutoDateRecognition/blob/main/Column_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.2.1

     |████████████████████████████████| 35.0MB 121kB/s 
     |████████████████████████████████| 890kB 57.6MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=d288cbe5ce07ad35479c4885402b0df4c2943722ec0cd596413ea2c383191c48
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
!pip install keras==2.0.7

     |████████████████████████████████| 276kB 9.3MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.7 which is incompatible.
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
!pip install faker

In [ ]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
import keras.backend as K
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

from faker import Faker
from lorem_text import lorem
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

## Creating dataset to be used for model building

In [ ]:
size = 5000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(size)
print(dataset[0])
print(human_vocab)
print(machine_vocab)
print(inv_machine_vocab)

100%|██████████| 5000/5000 [00:00<00:00, 14888.49it/s]

('9 may 1998', '1998-05-09')
{' ': 0, '.': 1, '/': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33, 'y': 34, '<unk>': 35, '<pad>': 36}
{'-': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10}
{0: '-', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '6', 8: '7', 9: '8', 10: '9'}


In [ ]:
dataset_l = [data[0] for data in dataset]

In [ ]:
dataset_l[:5]

['9 may 1998',
 '10.11.19',
 '9/10/70',
 'saturday april 28 1990',
 'thursday january 26 1995']

In [ ]:
num_l = list()
for i in range(size):
  num_l.append(str(random.randint(-10000, 10000)))

In [ ]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.corpus import gutenberg

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
emma = gutenberg.words('austen-emma.txt')
word_l = list(set(emma))[:size]
len(word_l)

5000

In [ ]:
emma_sent = gutenberg.sents('austen-emma.txt')
sent_l = list(x for x in emma_sent)[:size]
len(sent_l)

5000

In [ ]:
sent_s = [' '.join(sent) for sent in sent_l]
len(sent_s)

5000

In [ ]:
op_ls = list()
for i in range(0, size):
  op_ls.append(1)
for i in range(0, size*3):
  op_ls.append(0)

len(op_ls)

20000

In [ ]:
train_l = dataset_l + num_l + word_l + sent_s
len(train_l)

20000

In [ ]:
data_d = {'input': train_l, 'tag': op_ls}

In [ ]:
df = pd.DataFrame(data_d)
df.iloc[19000, :]

input    It would be the greatest pleasure to them , if...
tag                                                      0
Name: 19000, dtype: object

In [ ]:
df.head()

,input,tag
0,9 may 1998,1
1,10.11.19,1
2,9/10/70,1
3,saturday april 28 1990,1
4,thursday january 26 1995,1


In [ ]:
df.tail()

,input,tag
19995,"Elton .""",0
19996,He had done his duty and could return to his s...,0
19997,""" When Frank left us ,"" continued he , "" it wa...",0
19998,It has been completely unexpected .,0
19999,"That is , _I_ always had a strong persuasion h...",0


## Creating date classification model

In [ ]:
X = df['input']
Y = df['tag']

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 30
# This is fixed.
EMBEDDING_DIM = 25

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 10627 unique tokens.


In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [ ]:
sequences = tokenizer.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen = MAX_SEQUENCE_LENGTH)

In [ ]:
X[4]

'thursday january 26 1995'

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))  
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 25)            1250000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 30, 25)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               50400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 1,300,501
Trainable params: 1,300,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 5
batch_size = 64
history = model.fit(sequences_matrix, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 15300 samples, validate on 1700 samples
Epoch 1/5
15300/15300 [==============================] - 29s - loss: 0.0918 - acc: 0.9708 - val_loss: 0.0098 - val_acc: 0.9976
Epoch 2/5
15300/15300 [==============================] - 29s - loss: 0.0017 - acc: 0.9997 - val_loss: 0.0097 - val_acc: 0.9971
Epoch 3/5
15300/15300 [==============================] - 29s - loss: 3.6758e-04 - acc: 1.0000 - val_loss: 0.0125 - val_acc: 0.9971
Epoch 4/5
15300/15300 [==============================] - 29s - loss: 2.6571e-04 - acc: 1.0000 - val_loss: 0.0100 - val_acc: 0.9971
Epoch 5/5
15300/15300 [==============================] - 29s - loss: 4.2466e-04 - acc: 0.9999 - val_loss: 0.0115 - val_acc: 0.9971


In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_seq_matrix = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
accr = model.evaluate(test_seq_matrix, Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

3000/3000 [==============================] - 1s     
Test set
  Loss: 0.003
  Accuracy: 0.999


In [ ]:
preds = model.predict_classes(seq_matrix[:5])
preds

5/5 [==============================] - 0s


array([[1],
       [1],
       [1],
       [1],
       [1]], dtype=int32)

In [ ]:
test = random.sample(date_3, 10)
seqs = tokenizer.texts_to_sequences(test)
seq_matrix = sequence.pad_sequences(seqs, maxlen=MAX_SEQUENCE_LENGTH)
preds = model.predict_classes(seq_matrix)
preds = preds.tolist()

10/10 [==============================] - 0s


In [ ]:
preds.count([1])

10

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("class_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("class_model.h5")
print("Saved model to disk")

Saved model to disk


# Generating dataset for main project

In [ ]:
data_size = 4000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(data_size*3)
dataset_l = [data[0] for data in dataset]
dataset_l[:5]

100%|██████████| 12000/12000 [00:00<00:00, 17850.59it/s]


['jun 26 2003',
 '18 sep 1994',
 '4 january 1983',
 'wednesday february 13 1991',
 'thursday august 21 2003']

In [ ]:
len(dataset_l)

12000

In [ ]:
num_l = list()
for i in range(data_size):
  num_l.append(str(random.randint(-10000, 10000)))

In [ ]:
emma = gutenberg.words('austen-sense.txt')
word_l = list(set(emma))[:data_size]
len(word_l)

4000

In [ ]:
emma_sent = gutenberg.sents('melville-moby_dick.txt')
sent_l = list(x for x in emma_sent)[:data_size]
sent_s = [' '.join(sent) for sent in sent_l]
len(sent_s)

4000

In [ ]:
date_1 = dataset_l[:4000]
date_2 = dataset_l[4000:8000]
date_3 = dataset_l[8000:]
print("Date Sizes: ", len(date_1), len(date_2), len(date_3))

Date Sizes:  4000 4000 4000


In [ ]:
dataset_dict = {"Sentences": sent_s, "Date_1":date_1, "Words": word_l, "Date_2":date_2, "Numbers":num_l, "Date_3":date_3}

In [ ]:
data_df = pd.DataFrame(dataset_dict)
data_df.head()

,Sentences,Date_1,Words,Date_2,Numbers,Date_3
0,[ Moby Dick by Herman Melville 1851 ],jun 26 2003,affectionately,sunday august 31 2008,6141,sunday june 21 1998
1,ETYMOLOGY .,18 sep 1994,Madam,4 nov 1999,-5205,tuesday june 5 2007
2,( Supplied by a Late Consumptive Usher to a Gr...,4 january 1983,",",15 november 1988,-2748,monday november 19 1984
3,"The pale Usher -- threadbare in coat , heart ,...",wednesday february 13 1991,ornament,thursday march 24 1983,2114,saturday july 15 2000
4,He was ever dusting his old lexicons and gramm...,thursday august 21 2003,taxed,10 august 1985,5809,wednesday december 3 2014


In [ ]:
data_df.to_csv("dataset.csv")

In [ ]:
X = date_1 + date_2 + date_3 + sent_s + word_l + num_l
len(X)

24000

In [ ]:
with open('tokenizer.pickle', 'wb') as f:
  pickle.dump(tok, f, protocol=pickle.HIGHEST_PROTOCOL)